In [1]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.optimizers import SGD,Adam
import numpy as np
import matplotlib.pyplot as plt



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# batch_size = 128
# num_classes = 10 # 10 classes because we have 10 digits (0-9)
# epochs = 5

In [5]:
#perform data prep

In [17]:
model = Sequential()

model.add(keras.layers.BatchNormalization(input_shape=(448,448,3)))
model.add(Conv2D(16, (3,3), input_shape=(448,448,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(32, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(64, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(128, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(256, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(512, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(MaxPool2D((2,2)))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(1024, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(1024, (3,3), strides=1))
model.add(keras.layers.LeakyReLU(alpha=0.1))

model.add(keras.layers.BatchNormalization())
model.add(Conv2D(425, (1,1), strides=1, activation = "linear"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_12 (Batc (None, 448, 448, 3)       12        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 446, 446, 16)      448       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 446, 446, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 223, 223, 16)      0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 223, 223, 16)      64        
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 221, 221, 32)      4640      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 221, 221, 32)      0         
__________